In [2]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import re

In [3]:
#show evrything in dataframe
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
#open the corresponding file
#always choose .tsv file
df = pd.read_csv('L200_2025_Jan30_SpringMap-Map.tsv', sep='\t')

In [5]:
def format_to_orca(df: pd.DataFrame) -> pd.DataFrame:
    # Replace NaN values with '-'
    df = df.fillna('-')
    # Retain only columns up to and including 'HV Channel'
    if 'HV Channel' in df.columns:
        df = df.iloc[:, :df.columns.get_loc('HV Channel') + 1]
    # Rename specific columns
    rename_mapping = {' ': 'Det_name', 'HV Flange.1': 'HV Flange Pos'}
    df = df.rename(columns=rename_mapping)
    # Remove rows that contain 'String' in any column
    df = df[~df.apply(lambda row: row.astype(str).str.contains('String', na=False).any(), axis=1)]
    # Remove rows where all values are '-'
    df = df[~(df == '-').all(axis=1)]
    # Reset index
    df = df.reset_index(drop=True)
    # Limit to first 106 rows
    return df.iloc[:106]

In [6]:
def replace_862(value):
    if value == 'BF862':
        # Increment the counter for 862
        count = counter.get(value, 0) + 1
        counter[value] = count
        return f"{value}_{count:02}"
    return value

In [7]:
def cleandf(df):
  columns_to_drop = [
    'Cu rods', 'length from top of string', 'U(pper) or L(ower) row on HV connector',
    'min hv cable length', 'HV cable length', 'LMFE min cable length',
    'selected LMFE cable length', 'CC4 Position', 'Flashcam Channel #', 'HV Filter',
    'HV Filter Channel (0 left -> 7 right)', 'HV Filter Insulators', 'HV Filter Pulser',
    'HV Feedthrough'
  ]
  # Drop only existing columns to avoid errors
  df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
  df = df.fillna('--')  # Replace NaN with an empty string
  df = df.replace(['-','--','---'], '--')  # Replace '--', '-', and '---' with an empty string
  # Remove rows where all values are '-'
  df = df[~(df == '--').all(axis=1)]
  return df

In [8]:
def format_flashcam(df):
  # Extract all digits and expand into separate columns
  # make sure to include multi-digit numbers using regex
  df['Ext_Num'] = df['Flashcam'].str.findall(r'\d+')
  max_len = df['Ext_Num'].str.len().max()  # Determine max number of extracted numbers
  for i in range(max_len):
      df[f'Value_{i+1}'] = df['Ext_Num'].apply(lambda x: x[i] if i < len(x) else None)

  # Drop temporary list column
  df = df.drop(columns=['Ext_Num'])
  df = df.rename(columns={'Value_1': 'Falshcam chassis', 'Value_2': 'Flashcam slot', 'Value_3': 'Flashcam channel'})
  df['HV Flange'] = df['HV Flange'].astype(str).replace(r'\.', '', regex=True)
  df['CC4_F'] = df['CC4'] + '-' + df['CC4-Channel'].astype(str)
  # Remove rows where all values are '-'
  df = df[~(df == '--').all(axis=1)]
  return df

In [9]:
def categorize_entry(value):
    if value.startswith("BF862"):
        return "--"
    elif value.startswith("P"):
        return "ppc"
    elif value.startswith("B") and not value.startswith("BF862"):
        return "bege"
    elif value.startswith("V"):
        return "icpc"
    return "--"  # Default case if none of the conditions matc

In [10]:
def add_entry_to_json(df: pd.DataFrame, json_file: str):
    """
    Adds entries from a DataFrame to a JSON file following the specified format.

    Parameters:
    df (pd.DataFrame): DataFrame containing the new data.
    json_file (str): Path to the JSON file.
    """

    # Load existing JSON data or initialize an empty dictionary
    try:
        with open(json_file, 'r') as file:
            data = json.load(file)
    except (FileNotFoundError, json.JSONDecodeError):
        data = {}

    # Iterate over the DataFrame rows and add entries to JSON
    for _, row in df.iterrows():
        key = row['Det_name']  # Unique identifier for the entry (e.g., 'BF862_1', 'BF862_2', etc.)

        data[key] = {
            "system": str("ged"),
            "string": {
                "number": str(row.get("String", "--")),
                "position": str(row.get("Position", "--"))
            },
            "daq": {
                "board_ch": str(row.get("Flashcam channel", "--")),
                "adc_serial": str(row.get("ADC Serial", "--")),
                "board_slot": str(row.get("Flashcam slot", "--")),
                "board_id": str(row.get("Flashcam Address", "--")),
                "crate": str(row.get("Falshcam chassis", "--"))
            },
            "high_voltage": {
                "board_chan": str(row.get("HV Channel", "--")),
                "cable": str(row.get("HV Cable", "--")),
                "board_slot": str(row.get("HV Module", "--")),
                "flange_id": str(row.get("HV Flange", "--")),
                "flange_pos": str(row.get("HV Flange Pos", "--")),
                "crate": str('0')
            },
            "det_type": str(row.get("Det Type", "--")),
            "electronics": {
                "fanout_card": str(row.get("HE Cable", "--")),
                "lmfe_id": str(row.get("LMFE", "--")),
                "raspberrypi": str(row.get("RPi", "--")),
                "cc4_ch": str(row.get("CC4_F", "--")),
                "head_card_ana": str(row.get("Flange", "--")),
                "head_card_dig": str(row.get("HE Addr", "--"))
            }
        }

    # Write back to JSON file
    with open(json_file, 'w') as file:
        json.dump(data, file, indent=4)

    print(f"Entries successfully added to {json_file}")


In [11]:
# add board id with mapping
# Define mapping dictionary
mapping = {
    ('0', '0'): '0x120',
    ('0', '1'): '0x410',
    ('0', '2'): '0x420',
    ('0', '3'): '0x430',
    ('0', '4'): '0x440',
    ('0', '5'): '0x450',
    ('0', '6'): '0x460',
    ('0', '7'): '0x470',
    ('0', '8'): '0x480',
    ('0', '9'): '0x490',
    ('0', '10'): '0x4A0',
    ('0', '11'): '0x4B0',
    ('0', '12'): '0x4C0',
    ('1', '0'): '0x110',
    ('1', '1'): '0x310',
    ('1', '2'): '0x320',
    ('1', '3'): '0x330',
    ('1', '4'): '0x340',
    ('1', '5'): '0x350',
    ('1', '6'): '0x360'
    }

In [12]:
# Counter dictionary to track occurrences
counter = {}
# Apply the function to the DataFrame
df=format_to_orca(df)
df = df.map(replace_862)
df = cleandf(df)
df = format_flashcam(df)
df = cleandf(df)

In [13]:
df['Flashcam Address'] = df.apply(lambda row: mapping.get((row['Falshcam chassis'], row['Flashcam slot']), '--'), axis=1)


In [14]:
df["Det Type"] = df["Det_name"].astype(str).apply(categorize_entry)

In [15]:
df = df[df['Det_name'] != '--'].reset_index(drop=True)

In [16]:
#df

In [17]:
#open empty json file
# Define the file path
file_path = 'Automated_GeDet.json'

# Create an empty dictionary
data = {}

# Write the empty dictionary to the JSON file
with open(file_path, 'w') as f:
    json.dump(data, f)


In [18]:
add_entry_to_json(df,'Automated_GeDet.json')

Entries successfully added to Automated_GeDet.json


In [19]:
df

,Det_name,String,Position,HV Flange,LMFE,CC4-Channel,CC4,Flange,HE Addr,RPi,HE Cable,Flashcam,HV Flange Pos,HV Module,HV Channel,Falshcam chassis,Flashcam slot,Flashcam channel,CC4_F,Flashcam Address,Det Type
0,BF862_01,--,--,B28,--,0,A4,S3,01,--,15,1G4-1,Y,--,--,1,4,1,A4-0,0x340,--
1,BF862_02,--,--,B29,--,1,A4,S3,01,--,15,1G4-2,Y,--,--,1,4,2,A4-1,0x340,--
2,BF862_03,--,--,B30,--,2,A4,S3,01,--,15,1G4-3,Y,--,--,1,4,3,A4-2,0x340,--
3,BF862_04,--,--,B31,--,3,A4,S3,01,--,15,1G4-4,Y,--,--,1,4,4,A4-3,0x340,--
4,BF862_05,--,--,B32,--,4,A4,S3,01,--,15,1G4-5,Y,--,--,1,4,5,A4-4,0x340,--
5,BF862_06,--,--,B33,--,5,A4,S3,01,--,15,1G4-0,Y,--,--,1,4,0,A4-5,0x340,--
6,BF862_07,--,--,B34,--,6,A4,S3,01,--,15,1G5-1,Y,--,--,1,5,1,A4-6,0x350,--
7,BF862_08,--,--,B35,--,--,--,--,--,--,--,1G5-2,Y,--,--,1,5,2,-----,0x350,--
8,BF862_09,--,--,B36,--,--,--,--,--,--,--,1G5-3,Y,--,--,1,5,3,-----,0x350,--
9,BF862_10,--,--,--,--,--,--,--,--,--,--,1G5-4,--,--,--,1,5,4,-----,0x350,--
